In [25]:
# Import Packages
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

In [64]:
# Folder path
folder_path = r'C:\MIDS\ADS-506-Applied_Time_Series_Analysis\Bitcoin_time_series_analysis_and_forecast\data'

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Create a dictionary to store DataFrames
dataframes = {}

# Read each CSV file into a DataFrame and store it in the dictionary
for csv_file in csv_files:
    file_name = os.path.basename(csv_file)  # Extract the file name from the path
    short_name = os.path.splitext(file_name)[0]  # Remove the file extension
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    dataframes[short_name] = df

# Rename and create new DataFrames with shorter names
inflation_df = dataframes['5-Year Breakeven Inflation Rate - 2018-2021']
brent_oil_df = dataframes['Crude Oil Prices Brent - Europe - 2018-2021']
wti_oil_df = dataframes['Crude Oil WTI Futures Historical Data - 2018-2021']
federal_funds_df = dataframes['Daily Federal Funds Rate from 2018-2021']
treasury_rates_df = dataframes['daily-treasury-rates - 2018-2021']
crypto_df = dataframes['Digital_Currency']
djia_df = dataframes['Dow Jones Industrial Average-DJIA - 2018-2021']
uncertainty_df = dataframes['Economic Policy Uncertainty Index for United States - 2018-2021']

# Convert date columns to datetime format for each DataFrame
inflation_df['DATE'] = pd.to_datetime(inflation_df['DATE'])
brent_oil_df['DATE'] = pd.to_datetime(brent_oil_df['DATE'])
wti_oil_df['Date'] = pd.to_datetime(wti_oil_df['Date'])
federal_funds_df['DATE'] = pd.to_datetime(federal_funds_df['DATE'])
treasury_rates_df['Date'] = pd.to_datetime(treasury_rates_df['Date'])
crypto_df['Unnamed: 0'] = pd.to_datetime(crypto_df['Unnamed: 0'])
djia_df['DATE'] = pd.to_datetime(djia_df['DATE'])
uncertainty_df['DATE'] = pd.to_datetime(uncertainty_df['DATE'])

# Rename all date columns to "Date" in all DataFrames
inflation_df.rename(columns={'DATE': 'Date'}, inplace=True)
brent_oil_df.rename(columns={'DATE': 'Date'}, inplace=True)
wti_oil_df.rename(columns={'Date': 'Date'}, inplace=True)
federal_funds_df.rename(columns={'DATE': 'Date'}, inplace=True)
treasury_rates_df.rename(columns={'Date': 'Date'}, inplace=True)
crypto_df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
djia_df.rename(columns={'DATE': 'Date'}, inplace=True)
uncertainty_df.rename(columns={'DATE': 'Date'}, inplace=True)

# Merge all DataFrames on the 'Date' column
merged_df = pd.merge(inflation_df, brent_oil_df, on='Date', how='outer')
merged_df = pd.merge(merged_df, wti_oil_df, on='Date', how='outer')
merged_df = pd.merge(merged_df, federal_funds_df, on='Date', how='outer')
merged_df = pd.merge(merged_df, treasury_rates_df, on='Date', how='outer')
merged_df = pd.merge(merged_df, crypto_df, on='Date', how='outer')
merged_df = pd.merge(merged_df, djia_df, on='Date', how='outer')
merged_df = pd.merge(merged_df, uncertainty_df, on='Date', how='outer')

# Sort the DataFrame by the 'Date' column
merged_df.sort_values(by='Date', inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Print all columns in the merged DataFrame
for col in merged_df.columns:
    print(col)

Date
T5YIE
DCOILBRENTEU
Price
Open
High
Low
Vol.
Change %
DFF
4 WEEKS BANK DISCOUNT
4 WEEKS COUPON EQUIVALENT
8 WEEKS BANK DISCOUNT
8 WEEKS COUPON EQUIVALENT
13 WEEKS BANK DISCOUNT
13 WEEKS COUPON EQUIVALENT
26 WEEKS BANK DISCOUNT
26 WEEKS COUPON EQUIVALENT
52 WEEKS BANK DISCOUNT
52 WEEKS COUPON EQUIVALENT
open_SAR
open_USD
high_SAR
high_USD
low_SAR
low_USD
close_SAR
close_USD
volume
DJIA
USEPUINDXD


In [52]:
inflation_df.head(1)

,Date,T5YIE
0,2018-05-07,2.11


In [53]:
brent_oil_df.head(1)

,Date,DCOILBRENTEU
0,2018-05-08,74.16


In [54]:
wti_oil_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-01-29,52.2,52.15,53.25,51.93,410.91K,-0.27%


In [55]:
federal_funds_df.head(1)

,Date,DFF
0,2018-05-07,1.7


In [56]:
treasury_rates_df.head(1)

,Date,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
0,2018-01-02,1.27,1.29,NaN,NaN,1.42,1.44,1.58,1.61,1.78,1.83


In [57]:
crypto_df.head(1)

,Date,open_SAR,open_USD,high_SAR,high_USD,low_SAR,low_USD,close_SAR,close_USD,volume
0,2021-01-30,128437.248512,34246.28,131012.7232,34933.0,123106.88,32825.0,128333.212416,34218.54,43072


In [58]:
djia_df.head(1)

,Date,DJIA
0,2018-05-07,24357.32


In [59]:
uncertainty_df.head(1)

,Date,USEPUINDXD
0,2018-05-07,96.74
